# Playing around with the Spotify API

In [1]:
import pandas as pd
import numpy as np
import os
import sys
import spotipy
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation


Using TensorFlow backend.


In [2]:
'''
define the API connection - usually log in from here as well...
'''

spotify=spotipy.Spotify()


In [3]:
''' 
luke i changed ur function down below the index 1 was wrong so i put it as  0 and it giving
way better results, chur...
'''
def get_uri(artist):
    results = spotify.search(q = "artist:" + artist, type = "artist")
    artist_id = results['artists']['items'][0]['uri'] # changed from '1' to '0' ??
   
    return(artist_id)

get_uri("Beyonce")
#A function that returns artist 'uri' as a string. 
#By most matching (first result)

'spotify:artist:6vWDO969PvNqNYHIOW5v0m'

In [15]:
''' lets try get related song '''

artst = 'KISS'
related = spotify.artist_related_artists(get_uri(artst))
rels = []
for artist in related['artists']:
    rels.append(artist['name'])


# APP that takes you on a musical journey
It does this by moving to the least similar artist from each similar artist (the artist with the least back-links). Iteration expects you to get out of the genre the fastest of any other path

In [17]:
# get related artists:

def related_artists(name):
    related = spotify.artist_related_artists(get_uri(name))
    rels = []
    for artist in related['artists']:
        rels.append(artist['name'])
    return(rels)

In [63]:
related_artists('Future')

['Rich Homie Quan',
 '2 Chainz',
 'Young Thug',
 'Waka Flocka Flame',
 'Yo Gotti',
 'Migos',
 'French Montana',
 'Gucci Mane',
 'Meek Mill',
 'Travis Scott',
 'K CAMP',
 'Young Dolph',
 'A$AP Ferg',
 'Chief Keef',
 'Jeezy',
 'Juicy J',
 '21 Savage',
 'DJ Drama',
 'DJ Khaled',
 'Rick Ross']

In [19]:
from scipy import spatial

# to get cosine similarity...

In [58]:
# roll the predictions one forward and store each vector...
'''
Uncomment this out later:::

relates = [] # empty list to append with future 
for i in RelatedArtists:
    
    relates.append(related_artists(i) )
    
'''

# then create a bag of words vector for each proposed artist

BOW = np.unique(relates).flatten()
    
vecs = np.zeros((len(relates), len(BOW)))  # unique artists as columns...

for i in range(len(relates)):
    
    # turn each into a vector of 0 and 1s
    
    for art in relates[i]:
        # this loops over each artist:
        idx = np.where(BOW == art)[0][0]
        # set that idx number = to 1
        vecs[i,idx] = 1
    
    
# then calculate cosine similarity and select the lowest similarity artist and print them
sims = np.zeros((len(relates),len(relates)))
sim_means = []
for i in range(len(vecs)):
    # loop over each vector
    for j in range(len(vecs)):
        # do this again because you want to get similarity between each matrix...
        sims[i,j] = 1 - spatial.distance.cosine(vecs[i],vecs[j])
    # calculate row wise means:
    sim_means.append(np.mean(sims[i] ))
    

lowest_idx = np.where(sim_means == min(sim_means) )[0][0]

# print artists name:

print(RelatedArtists[lowest_idx])

#excellent it works...

#now turn into a function

Cheap Trick


In [68]:
def next_artist(artist, artists = []):
    ''' artists is the storage of past people that have been in there'''
    RelatedArtists = related_artists(artist)
    relates = [] # empty list to append with future 
    
    for i in RelatedArtists:
        relates.append(related_artists(i) )
    
    # create a bag of words - to index each artist in:
    
    BOW = np.unique(relates).flatten() 
    vecs = np.zeros((len(relates), len(BOW)))  # unique artists as columns...

    for i in range(len(relates)):
        # turn each into a vector of 0 and 1
        for art in relates[i]:
            # this loops over each artist:
            idx = np.where(BOW == art)[0][0]
            # set that idx number = to 1
            vecs[i,idx] = 1
    
    
    # then calculate cosine similarity and select the lowest similarity artist and print them
    sims = np.zeros((len(relates),len(relates)))
    sim_means = []
    for i in range(len(vecs)):
        # loop over each vector
        for j in range(len(vecs)):
            # do this again because you want to get similarity between each matrix...
            sims[i,j] = 1 - spatial.distance.cosine(vecs[i],vecs[j])
        # calculate row wise means:
        sim_means.append(np.mean(sims[i] ))
    
    # find the lowest scoring similarity artist
    lowest_idx = np.where(sim_means == min(sim_means) )[0][0]
    #print(lowest_idx)
    # return artists name:
    

    if (RelatedArtists[lowest_idx] in artists) == True:
        
    
    
    print(RelatedArtists[lowest_idx])
    
    
    return(RelatedArtists[lowest_idx])
    

In [65]:
# try that out:
next_artist('future')
# cool it works -- need to make a function to roll over with iters

12


'A$AP Ferg'

In [66]:
def roll_out(artist, iters, artists = []):
    artists.append(artist)
    for i in range(iters):
        artist = next_artist(artist, artists)
        artists.append(artist)
        
    return(artist, artists)

In [67]:
roll_out('A$AP Ferg',2)

6
10


'Chief Keef'

In [70]:
roll_out('Waka Flocka Flame',2)

10
Chief Keef
13
A$AP Ferg


'A$AP Ferg'

In [72]:
x = [3,2,1,4,5]

3 in x


True